<a href="https://colab.research.google.com/github/wadeaT/Cloud-Computing-Course/blob/main/SearchEngineTurtles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests beautifulsoup4
!pip install firebase
!pip install firebase-admin

In [ ]:
# Import necessary libraries
from google.colab import drive, output
from IPython.display import HTML, Javascript
from IPython.display import clear_output
from firebase_admin import credentials
from firebase_admin import firestore
from nltk.chat.util import Chat, reflections
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import firebase_admin
import IPython
import random
import json
import os
import re
import requests
import nltk
import logging
from typing import Optional, List, Tuple, Dict
from dataclasses import dataclass

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:

# Setup directory
directory = '/content/Cloud-Computing-Course'

# Check if directory exists and remove if it does
if os.path.exists(directory):
    !rm -r "$directory"

# Change to /content directory and clone repository
os.chdir('/content')
!git clone "https://github.com/wadeaT/Cloud-Computing-Course.git"

# Mount Google Drive
drive.mount('/content/drive')

# Change to HW2 directory and run index notebook
%cd "/content/drive/My Drive/Colab Notebooks/HW2"
%run index.ipynb

# Get index from database
index = get_index_from_db()

Cloning into 'Cloud-Computing-Course'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 37 (delta 5), reused 6 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 435.29 KiB | 2.30 MiB/s, done.
Resolving deltas: 100% (5/5), done.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/HW2


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Processing page 1/50: https://cloud.google.com/products/compute
Processing page 2/50: https://cloud.google.com/compute/docs/disks/performance#type_comparison
Processing page 3/50: https://cloud.google.com/compute/disks-image-pricing#localssdpricing
Processing page 4/50: https://cloud.google.com/compute/docs/sustained-use-discounts?hl=en
Processing page 5/50: https://cloud.google.com/vmware-engine/
Processing page 6/50: https://cloud.google.com/vmware-engine?hl=en
Processing page 7/50: https://cloud.google.com/compute/docs/nodes/sole-tenant-nodes/
Processing page 8/50: https://cloud.google.com/compute/docs/general-purpose-machines#c4a_series
Processing page 9/50: https://cloud.google.com/compute/docs/instances/create-start-instance
Processing page 10/50: https://cloud.google.com/kubernetes-engine/docs/tutorials/hello-app
Processing page 11/50: https://cloud.google.com/compute/docs/regions-zones/viewing-regions-zones
Processing page 12/50: https://cloud.google.com/compute/docs/general-pu

In [ ]:
from typing import Optional, List, Tuple, Dict, Any
from dataclasses import dataclass
from datetime import datetime
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from google.colab import drive, output
from IPython.display import HTML, Javascript, clear_output
import logging
import requests
import re
import json
import random
import asyncio
import aiohttp
import nest_asyncio
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor
from math import log

# Enable nested event loops
nest_asyncio.apply()

@dataclass
class ScreenContent:
    html: str
    css: Optional[str] = None
    js: Optional[str] = None

class CacheManager:
    def __init__(self, cache_timeout: int = 3600):
        self.cache_timeout = cache_timeout
        self.title_cache = {}
        self.paragraph_cache = {}
        self.last_updated = {}

    @lru_cache(maxsize=1000)
    def get_cached_data(self, key: str, cache_type: str) -> Optional[str]:
        cache = self.title_cache if cache_type == 'title' else self.paragraph_cache
        if key in cache and (datetime.now() - self.last_updated[key]).seconds < self.cache_timeout:
            return cache[key]
        return None

    def set_cached_data(self, key: str, value: str, cache_type: str):
        cache = self.title_cache if cache_type == 'title' else self.paragraph_cache
        cache[key] = value
        self.last_updated[key] = datetime.now()

class HTTPClient:
    def __init__(self, max_retries: int = 3, timeout: int = 10):
        self.max_retries = max_retries
        self.timeout = timeout
        self._session = None

    @property
    def session(self):
        if self._session is None:
            self._session = aiohttp.ClientSession()
        return self._session

    async def fetch_async(self, url: str) -> str:
        for _ in range(self.max_retries):
            try:
                async with self.session.get(url, timeout=self.timeout) as response:
                    return await response.text()
            except Exception as e:
                if _ == self.max_retries - 1:
                    raise
                await asyncio.sleep(1)
        return ""

    async def close(self):
        if self._session:
            await self._session.close()
            self._session = None

class ScreenManager:
    def __init__(self, base_url: str, http_client: HTTPClient):
        self.base_url = base_url.rstrip('/')
        self.http_client = http_client
        self.logger = logging.getLogger(__name__)
        self.loop = asyncio.get_event_loop()

    async def fetch_screen_content(self, screen_name: str, css_path: Optional[str] = None, js_path: Optional[str] = None) -> ScreenContent:
        html_path = f"{self.base_url}/HW2/GUI/{screen_name}.html"
        tasks = [self.http_client.fetch_async(html_path)]

        if css_path:
            tasks.append(self.http_client.fetch_async(f"{self.base_url}/{css_path}"))
        if js_path:
            tasks.append(self.http_client.fetch_async(f"{self.base_url}/{js_path}"))

        try:
            results = await asyncio.gather(*tasks)
            return ScreenContent(
                html=results[0],
                css=results[1] if css_path else None,
                js=results[2] if js_path else None
            )
        except Exception as e:
            self.logger.error(f"Error fetching screen content: {str(e)}")
            return ScreenContent("")

    def switch_to_screen(self, screen_name: str, css_path: Optional[str] = None, js_path: Optional[str] = None):
        clear_output(wait=True)
        try:
            content = self.loop.run_until_complete(
                self.fetch_screen_content(screen_name, css_path, js_path)
            )
            self._display_screen(content)
        except Exception as e:
            self.logger.error(f"Error switching to screen {screen_name}: {str(e)}")
            self._display_error_screen()

    def _display_screen(self, screen: ScreenContent):
        try:
            if screen.css:
                display(HTML(f'<style>{screen.css}</style>'))
            if screen.html:
                display(HTML(screen.html))
            if screen.js:
                display(Javascript(screen.js))
        except Exception as e:
            self.logger.error(f"Error displaying screen: {str(e)}")
            self._display_error_screen()

    def _display_error_screen(self):
        error_html = """
        <div style="text-align: center; padding: 20px;">
            <h2>Error Loading Screen</h2>
            <p>We encountered a problem loading the requested screen. Please try again.</p>
            <button onclick="notebook.back_to_home_page()"
                    style="padding: 10px; background-color: #007bff; color: white;
                           border: none; border-radius: 5px; cursor: pointer;">
                Return to Home
            </button>
        </div>
        """
        display(HTML(error_html))

class SearchEngine:
    def __init__(self, base_url: str, firebase_url: str):
        self.base_url = base_url
        self.firebase_url = firebase_url
        self.http_client = HTTPClient()
        self.screen_manager = ScreenManager(base_url, self.http_client)
        self.cache_manager = CacheManager()
        self.setup_logging()
        self.setup_nlp()
        self.loop = asyncio.get_event_loop()

    def setup_logging(self):
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
        )
        self.logger = logging.getLogger(__name__)

    def setup_nlp(self):
        self.stemmer = PorterStemmer()
        self.stop_words = set(stopwords.words('english'))
        self.stop_words.update({'ha', 'thi', 'skip', '-', '&'})

    def handle_search(self, query: str):
        if not query.strip():
            return

        try:
            self.loop.run_until_complete(self._async_handle_search(query))
        except Exception as e:
            self.logger.error(f"Search error: {str(e)}")
            self._display_error_message("An error occurred while searching.")

    async def _async_handle_search(self, query: str):
        loading_message = random.choice([
            "Searching through the database...",
            "Processing your request...",
            "Finding relevant results...",
            "Almost there..."
        ])

        display(HTML(f"""
            <script>
                document.getElementById("searchbartext").value = "{query}";
                document.getElementById("loading").innerHTML = "{loading_message}";
            </script>
        """))

        try:
            results = await self.search(query, index)
            self._display_results(
                [result['title'] for result in results],
                [result['url'] for result in results],
                [result['description'] for result in results],
                sum(result['relevance'] for result in results)
            )
            await self.update_statistics(query)
        except Exception as e:
            self.logger.error(f"Search error: {str(e)}")
            self._display_error_message("An error occurred while searching.")

    def process_search_query(self, query: str) -> List[str]:
        # Clean and normalize the query
        query = re.sub(r'[^\w\s-]', ' ', query.lower())
        query = self._expand_abbreviations(query)

        # Tokenize and process
        tokens = word_tokenize(query)

        # Filter and stem tokens
        processed_tokens = []
        for token in tokens:
            if (
                token not in self.stop_words
                and len(token) > 1  # Skip single characters
                and not token.isnumeric()  # Skip pure numbers
            ):
                stemmed = self.stemmer.stem(token)
                if stemmed:
                    processed_tokens.append(stemmed)

        return processed_tokens

    def _expand_abbreviations(self, text: str) -> str:
        abbreviations = {
            'redhat': 'red hat',
            'gcp': 'google cloud platform',
            'gc': 'google cloud',
            'aws': 'amazon web services',
            'k8s': 'kubernetes',
            'vm': 'virtual machine',
            'vms': 'virtual machines',
            'api': 'application programming interface',
            'apis': 'application programming interfaces',
            'db': 'database',
            'dbs': 'databases',
            'ai': 'artificial intelligence',
            'ml': 'machine learning',
            'tpu': 'tensor processing unit',
            'gpu': 'graphics processing unit',
            'cdn': 'content delivery network',
            'dns': 'domain name system',
            'lb': 'load balancer',
            'vpc': 'virtual private cloud',
            'iam': 'identity and access management',
            'ci': 'continuous integration',
            'cd': 'continuous delivery',
            'devops': 'development operations',
            'sre': 'site reliability engineering',
            'saas': 'software as a service',
            'paas': 'platform as a service',
            'iaas': 'infrastructure as a service',
            'os': 'operating system'
        }
        for abbr, full in abbreviations.items():
            text = text.replace(abbr, full)
        return text

    async def search(self, query: str, index: dict) -> List[Dict[str, Any]]:
        tokens = self.process_search_query(query)
        if not tokens:
            return []

        results = []
        seen_urls = set()
        token_scores = {}

        # Get total number of documents
        all_doc_ids = set()
        for token_info in index.values():
            for doc_info in token_info.values():
                if 'url' in doc_info:
                    all_doc_ids.add(doc_info['url'])
        total_docs = len(all_doc_ids)

        # Calculate TF-IDF scores
        for token in tokens:
            if token in index:
                # Calculate IDF (Inverse Document Frequency)
                docs_with_term = len(index[token])
                idf = log((1 + total_docs) / (1 + docs_with_term)) + 1

                # Calculate TF (Term Frequency) for each document
                for doc_id, doc_info in index[token].items():
                    if 'url' not in doc_info:
                        continue

                    url = doc_info['url']
                    if url not in token_scores:
                        token_scores[url] = 0

                    # TF-IDF scoring with term frequency normalization
                    tf = 1 + log(doc_info.get('counter', 1))
                    token_scores[url] += tf * idf

        # Get content for top-scoring documents
        tasks = []
        for url in sorted(token_scores.keys(), key=lambda x: token_scores[x], reverse=True)[:10]:
            if url not in seen_urls:
                seen_urls.add(url)
                tasks.append(self.get_page_content(url))

        contents = await asyncio.gather(*tasks)

        for url, (title, description) in zip(list(seen_urls)[:10], contents):
            results.append({
                'url': url,
                'title': title,
                'description': description,
                'relevance': token_scores[url]
            })

        return results

    async def get_page_content(self, link: str) -> Tuple[str, str]:
        cached_title = self.cache_manager.get_cached_data(link, 'title')
        cached_paragraph = self.cache_manager.get_cached_data(link, 'paragraph')

        if cached_title and cached_paragraph:
            return cached_title, cached_paragraph

        try:
            html_content = await self.http_client.fetch_async(link)
            soup = BeautifulSoup(html_content, 'html.parser')

            title = self._extract_title(soup)
            paragraph = self._extract_first_paragraph(soup)

            self.cache_manager.set_cached_data(link, title, 'title')
            self.cache_manager.set_cached_data(link, paragraph, 'paragraph')

            return title, paragraph
        except Exception as e:
            self.logger.error(f"Error fetching content from {link}: {str(e)}")
            return "Error loading title", "Error loading content"

    def _extract_title(self, soup: BeautifulSoup) -> str:
        for selector in ['title', 'meta[property="og:title"]', 'h1']:
            element = soup.select_one(selector)
            if element:
                if selector == 'meta[property="og:title"]':
                    title = element.get('content', '')
                else:
                    title = element.get_text()
                if title:
                    return self._clean_and_truncate_text(title)
        return "No title found"

    def _extract_first_paragraph(self, soup: BeautifulSoup) -> str:
        for p in soup.find_all(['p', 'div']):
            text = self._clean_and_truncate_text(p.get_text())
            if len(text) > 50:
                return text
        return "No meaningful content found"

    def _clean_and_truncate_text(self, text: str, max_length: int = 200) -> str:
        text = ' '.join(text.split())
        text = re.sub(r'[^\w\s-]', '', text)
        if len(text) > max_length:
            text = text[:max_length-3] + "..."
        return text

    def _display_results(self, titles, links, texts, times_appeared):
        # Escape special characters in the strings
        titles_escaped = [re.sub(r'(["\\\'])', r'\\\1', title) for title in titles]
        links_escaped = [re.sub(r'(["\\\'])', r'\\\1', link) for link in links]
        texts_escaped = [re.sub(r'(["\\\'])', r'\\\1', text) for text in texts]

        # Join with proper JSON formatting
        titles_js = ', '.join(f'"{title}"' for title in titles_escaped)
        links_js = ', '.join(f'"{link}"' for link in links_escaped)
        texts_js = ', '.join(f'"{text}"' for text in texts_escaped)

        # Handle empty results
        if not titles:
            display(HTML("""
                <script>
                    document.getElementById("loading").innerHTML = "";
                    document.getElementById("results").innerHTML =
                        '<div class="no-results">No results found. Try different keywords.</div>';
                </script>
            """))
            return

        display(HTML(f"""
            <script>
                document.getElementById("loading").innerHTML = "";
                renderResults_title_link(
                    [{titles_js}],
                    [{links_js}],
                    [{texts_js}],
                    {times_appeared}
                );
            </script>
        """))

    def _display_error_message(self, message: str):
        display(HTML(f"""
            <script>
                document.getElementById("loading").innerHTML = "";
                document.getElementById("results").innerHTML =
                    '<div class="error-message">{message}</div>';
            </script>
        """))

    async def update_statistics(self, query: str):
        try:
            stats_url = f"{self.firebase_url}statistics.json"
            async with aiohttp.ClientSession() as session:
                async with session.get(stats_url) as response:
                    stats = await response.json() or {}

                existing_stat = None
                stat_id = None
                for sid, stat in stats.items():
                    if stat.get('term') == query:
                        existing_stat = stat
                        stat_id = sid
                        break

                if existing_stat:
                    existing_stat['count'] += 1
                    existing_stat['timestamp'] = datetime.now().isoformat()
                    async with session.patch(
                        f"{self.firebase_url}statistics/{stat_id}.json",
                        json=existing_stat
                    ) as response:
                        await response.json()
                else:
                    new_stat = {
                        'term': query,
                        'count': 1,
                        'timestamp': datetime.now().isoformat()
                    }
                    async with session.post(
                        f"{self.firebase_url}statistics.json",
                        json=new_stat
                    ) as response:
                        await response.json()

        except Exception as e:
            self.logger.error(f"Error updating statistics: {str(e)}")

    async def get_statistics(self) -> List[Dict[str, Any]]:
        try:
            stats_url = f"{self.firebase_url}statistics.json"
            async with aiohttp.ClientSession() as session:
                async with session.get(stats_url) as response:
                    stats = await response.json() or {}
                    return sorted(
                        stats.values(),
                        key=lambda x: x.get('count', 0),
                        reverse=True
                    )
        except Exception as e:
            self.logger.error(f"Error fetching statistics: {str(e)}")
            return []

    async def clear_statistics(self):
        try:
            stats_url = f"{self.firebase_url}statistics.json"
            async with aiohttp.ClientSession() as session:
                async with session.delete(stats_url) as response:
                    await response.json()
        except Exception as e:
            self.logger.error(f"Error clearing statistics: {str(e)}")

def initialize_search_engine(base_url: str, firebase_url: str) -> SearchEngine:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
    nltk.download('wordnet', quiet=True)

    search_engine = SearchEngine(base_url, firebase_url)

    def handle_search_callback(query):
        search_engine.handle_search(query)

    output.register_callback('notebook.receive_search_string', handle_search_callback)

    def handle_switch_to_home():
        search_engine.screen_manager.switch_to_screen(
            'index',
            css_path="HW2/GUI/style.css",
            js_path="HW2/GUI/script.js"
        )

    def handle_switch_to_admin():
        search_engine.screen_manager.switch_to_screen(
            'admin',
            css_path="HW2/GUI/style.css",
            js_path="HW2/GUI/script.js"
        )

    def handle_switch_to_statistics():
        search_engine.screen_manager.switch_to_screen(
            'statistics',
            css_path="HW2/GUI/style.css",
            js_path="HW2/GUI/script.js"
        )

    output.register_callback('notebook.back_to_home_page', handle_switch_to_home)
    output.register_callback('notebook.open_indexeditor_page', handle_switch_to_admin)
    output.register_callback('notebook.open_statistics_screen', handle_switch_to_statistics)
    output.register_callback('notebook.back_to_admin', handle_switch_to_admin)

    async def handle_clear_statistics():
        await search_engine.clear_statistics()
        search_engine.screen_manager.switch_to_screen(
            'statistics',
            css_path="HW2/GUI/style.css",
            js_path="HW2/GUI/script.js"
        )

    async def handle_get_statistics():
        stats = await search_engine.get_statistics()
        display(HTML(f"""
            <script>
                document.getElementById("statisticsContainer").innerHTML =
                    '{json.dumps(stats)}';
            </script>
        """))

    output.register_callback('notebook.clear_statistics',
        lambda: asyncio.create_task(handle_clear_statistics()))
    output.register_callback('notebook.get_statistics',
        lambda: asyncio.create_task(handle_get_statistics()))

    return search_engine

async def cleanup(search_engine):
    await search_engine.http_client.close()

def verify_index_structure(index):
    if not isinstance(index, dict):
        return False

    for token, docs in index.items():
        if not isinstance(docs, dict):
            return False
        for doc_id, info in docs.items():
            if not isinstance(info, dict) or 'url' not in info or 'counter' not in info:
                return False
    return True

if __name__ == "__main__":
    base_url = "https://raw.githubusercontent.com/wadeaT/Cloud-Computing-Course/main"
    firebase_url = "https://turtlenimbus-d033a-default-rtdb.firebaseio.com/"

    try:
        search_engine = initialize_search_engine(base_url, firebase_url)

        # Verify index structure
        index = get_index_from_db()  # You need to implement this function
        if not verify_index_structure(index):
            print("Warning: Index structure is not correct")

        search_engine.screen_manager.switch_to_screen(
            'index',
            css_path="HW2/GUI/style.css",
            js_path="HW2/GUI/script.js"
        )
        print("Search engine initialized successfully!")
    except Exception as e:
        print(f"Error initializing search engine: {str(e)}")
        raise
    finally:
        # Ensure proper cleanup when the program exits
        loop = asyncio.get_event_loop()
        loop.run_until_complete(cleanup(search_engine))

<IPython.core.display.Javascript object>

Search engine initialized successfully!


ERROR:__main__:Search error: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

